In [1]:
from __future__ import annotations

import os
import sys
import time
import copy
import yaml
import logging
import argparse
from ase import Atoms
from ase.io import read

import numpy as np
import torch
import pytest

from ase import Atoms, units
from ase.md.langevin import Langevin
from ase.io import read, write
from ase.constraints import FixAtoms
from ase.calculators.calculator import Calculator, all_changes
from ase.stress import full_3x3_to_voigt_6_stress
from ase.calculators.singlepoint import SinglePointCalculator as sp

from tqdm import tqdm
import pytorch_lightning as pl

from matsciml.common.registry import registry
from matsciml.common.utils import radius_graph_pbc, setup_imports, setup_logging
from matsciml.datasets.transforms import (
    PeriodicPropertiesTransform,
    PointCloudToGraphTransform,
    FrameAveraging,
)
from matsciml.datasets.trajectory_lmdb import data_list_collater
from matsciml.lightning import MatSciMLDataModule
from matsciml.models.pyg import FAENet
from matsciml.models.base import ForceRegressionTask
from matsciml.models.utils.io import multitask_from_checkpoint
from matsciml.preprocessing.atoms_to_graphs import *


In [2]:
checkpoint_path = "/home/m3rg2000/Simulation/checkpoints-2024/FAENet_250k.ckpt"
Loaded_model = multitask_from_checkpoint(checkpoint_path)

In [3]:
a2g=AtomsToGraphs(max_neigh=200,
            radius=6,
            r_energy=False,
            r_forces=False,
            r_distances=False,
            r_edges=True,
            r_fixed=True,)
f_avg=FrameAveraging(frame_averaging="3D", fa_method="stochastic")

def convAtomstoBatch(atoms):
    data_obj=a2g.convert(atoms)
    Reformatted_batch={
        'cell' : data_obj.cell,
        'natoms' :  torch.Tensor([data_obj.natoms]).unsqueeze(0),
        'edge_index' : [data_obj.edge_index.shape],
        'cell_offsets': data_obj.cell_offsets,
        'y' : None,
        'force' : None, 
        'fixed' : [data_obj.fixed],
        'tags' : None,
        'sid' :None,
        'fid' : None,
        'dataset' : 'S2EFDataset',
        'graph' : data_list_collater([data_obj]),
    }
    Reformatted_batch=f_avg(Reformatted_batch)
    return Reformatted_batch


In [4]:

class FAENet_ASEcalculator(Calculator):
    """Simulation ASE Calculator"""

    implemented_properties = ["energy", "free_energy", "forces", "stress"]

    def __init__(
        self,
        model,
        **kwargs
    ):
        Calculator.__init__(self, **kwargs)
        self.results = {}

        self.model = model
        
        
    # pylint: disable=dangerous-default-value
    def calculate(self, atoms=None, properties=None, system_changes=all_changes):
        """
        Calculate properties.
        :param atoms: ase.Atoms object
        :param properties: [str], properties to be computed, used by ASE internally
        :param system_changes: [str], system changes since last calculation, used by ASE internally
        :return:
        """
        # call to base-class to set atoms attribute
        Calculator.calculate(self, atoms)

        # prepare data
        batch=convAtomstoBatch(atoms)

        # predict + extract data
        out = self.model(batch)
        energy = out['force_regression0']["energy"].detach().cpu().item()
        forces = out['force_regression0']["force"].detach().cpu().numpy()
        stress = out['force_regression0']["stress"].squeeze(0).detach().cpu().numpy()
        # store results
        E = energy
        stress= np.array([stress[0, 0],
                                   stress[1, 1],
                                   stress[2, 2],
                                   stress[1, 2],
                                   stress[0, 2],
                                   stress[0, 1]])
        self.results = {
            "energy": E,
            # force has units eng / len:
            "forces": forces,
            "stress" : stress,
        }


In [5]:
def get_density(atoms):
    amu_to_grams = 1.66053906660e-24  # 1 amu = 1.66053906660e-24 grams
    angstrom_to_cm = 1e-8  # 1 Å = 1e-8 cm
    mass_amu = atoms.get_masses().sum()
    mass_g = mass_amu * amu_to_grams    # Get the volume of the atoms object in cubic angstroms (Å³)
    volume_A3 = atoms.get_volume()
    volume_cm3 = volume_A3 * (angstrom_to_cm ** 3)  # 1 Å³ = 1e-24 cm³
    density = mass_g / volume_cm3
    
    return density





In [6]:
def run_simulation(calculator,atoms, pressure=1.01325, temperature=298, timestep=0.1, steps=10):
         
    # Define the temperature and pressure
    
    init_conf=atoms
    init_conf.set_calculator(calculator)
    # Initialize the NPT dynamics
    dyn = NPTBerendsen(init_conf, timestep=timestep * units.fs, temperature_K=temperature,
                       taut=100 * units.fs, pressure_au=pressure * units.bar,
                       taup=1000 * units.fs, compressibility_au=4.57e-5 / units.bar)
    
    density=[]
    angles=[]
    lattice_parameters=[]
    def write_frame():
        # dyn.atoms.write('md_FAENET_npt1.xyz', append=True)
        cell = dyn.atoms.get_cell()
        
        lattice_parameters.append(cell.lengths())  # Get the lattice parameters
        angles.append(cell.angles())  # Get the angles
        density.append(get_density(atoms))

    
    
    dyn.attach(write_frame, interval=1)
    dyn.run(steps)

    density = np.array(density)
    angles = np.array(angles)
    lattice_parameters = np.array(lattice_parameters)
    
    # Calculate average values
    avg_density = np.mean(density)
    avg_angles = np.mean(angles, axis=0)
    avg_lattice_parameters = np.mean(lattice_parameters, axis=0)
    
    return avg_density, avg_angles, avg_lattice_parameters


    


## Experimental Eval

In [7]:
cif_files_dir = "/home/m3rg2000/Simulation/amcsd"
output_file = "/home/m3rg2000/Simulation/Exp_task/cif_properties.csv"
import pandas as pd
# List to hold the data
data = []
calculator = FAENet_ASEcalculator(Loaded_model)
for folder in os.listdir(cif_files_dir):
    folder_path = os.path.join(cif_files_dir, folder)
    
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            try:
                file_path = os.path.join(folder_path, file)
                atoms = read(file_path)
                print(file_path)
                
                # Calculate density and cell lengths and angles
                density = get_density(atoms)
                cell_lengths_and_angles = atoms.get_cell_lengths_and_angles().tolist()

                avg_density, avg_angles, avg_lattice_parameters=run_simulation(calculator,atoms, pressure=1.01325, temperature=298, timestep=0.1, steps=10 )
                
                # Append the results to the data list
                data.append([folder+file[:-4], density] + cell_lengths_and_angles+[avg_density]+avg_lattice_parameters.tolist()+avg_angles.tolist())

            
                    
            except Exception as e:
                print(f"Error processing file {file}: {e}")
                continue

# Create a DataFrame
columns = ["Filename", "Exp_Density (g/cm³)", "Exp_a (Å)", "Exp_b (Å)", "Exp_c (Å)", "Exp_alpha (°)", "Exp_beta (°)", "Exp_gamma (°)"
           ,"Sim_Density (g/cm³)", "Sim_a (Å)", "Sim_b (Å)", "Sim_c (Å)", "Sim_alpha (°)", "Sim_beta (°)", "Sim_gamma (°)"]
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")


Error processing file 298_1_4.cif: 'Oa'
/home/m3rg2000/Simulation/amcsd/tobormorite/298_1_6.cif


/home/m3rg2000/miniconda3/envs/matsciml/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


Error processing file 298_1_6.cif: name 'NPTBerendsen' is not defined
/home/m3rg2000/Simulation/amcsd/tobormorite/298_1_1.cif
Error processing file 298_1_1.cif: name 'NPTBerendsen' is not defined
/home/m3rg2000/Simulation/amcsd/tobormorite/572_1_3.cif
Error processing file 572_1_3.cif: name 'NPTBerendsen' is not defined
Error processing file 297_1_2.cif: invalid spacegroup `C 1 2/c 1`, setting `None` not found in data base
/home/m3rg2000/Simulation/amcsd/tobormorite/298_26.6_5.cif
Error processing file 298_26.6_5.cif: name 'NPTBerendsen' is not defined
Error processing file .ipynb_checkpoints: Not a BundleTrajectory: /home/m3rg2000/Simulation/amcsd/tobormorite/.ipynb_checkpoints
Data saved to /home/m3rg2000/Simulation/Exp_task/cif_properties.csv
